## Create Geometry

In [1]:
import numpy as np
import nibabel as nib
# import itk
import json
import yaml
import glob
import os

In [2]:
# scans = ['056', '057', '116', '175', '253', '255', '257']
scans = ['010', '027', '221']

## Lung mask

In [5]:
out_dir = '/home/ahaas/data/0_input_simulation/geometries/'

def prepare_lung_mask(scan):
    atm = nib.load(f'/home/shared/Data/ATM22/train/masks/ATM_{scan}_0000_lobes_mask.nii.gz')
    atm_data = atm.get_fdata().astype(np.uint8)
    
    os.makedirs(f'{out_dir}/ATM_{scan}', exist_ok=True)
    for i in range(1, 6):
        lobe = np.where(atm_data == i, 1, 0).astype(np.uint8)
        np.save(f'{out_dir}/ATM_{scan}/lobe_{i}_simulation_space.npy', lobe)

In [6]:
for scan in scans:
    prepare_lung_mask(scan)

## Roots for airways and vessels

In [ ]:
base_dir = '/home/ahaas/data/0_input_simulation'

def markers2yml(scan, option='airways'):
    os.makedirs(f'{base_dir}/geometries/ATM_{scan}/', exist_ok=True)
    # out_file = f'{base_dir}/geometries/ATM_{scan}/ATM_{scan}_{option}_roots.yml'

    lobes = {}
    # weird_spacing
    with open(f'{base_dir}/ATM_preparation/roots/ATM_{scan}/ATM_{scan}_spacing.yml') as f:
        spacing = yaml.safe_load(f)

    p_min = np.array((spacing['x_min'], spacing['y_min'], spacing['z_min']))
    p_max = np.array((spacing['x_max'], spacing['y_max'], spacing['z_max']))
    size = np.array((spacing['x_out'], spacing['y_out'], spacing['z_out']))

    for lobe_idx in range(1, 6):
        roots = []
        marker_files = glob.glob(f'{base_dir}/ATM_preparation/roots/ATM_{scan}/{option}_roots/{lobe_idx}/*.mrk.json')
        for i, marker_file in enumerate(marker_files):
            with open(marker_file) as f:
                marker = json.load(f)
            points = marker['markups'][0]['controlPoints']
            point1 = np.array(points[0]['position'])
            point2 = np.array(points[1]['position'])
            point1 = (point1-p_min) / (p_max-p_min) * size # normalizing to [0, 1] and scaling to output size
            point2 = (point2-p_min) / (p_max-p_min) * size 
            direction = (point2-point1) / np.linalg.norm(point2-point1)
            roots.append({
                'name': f'root_{i}',
                'position': point1.tolist(),
                'direction': direction.tolist()
            })

        lobes[f'lobe{lobe_idx}'] = roots
        with open(f'{base_dir}/geometries/ATM_{scan}/lobe_{lobe_idx}_{option}s_roots.yml', 'w') as f:
            yaml.dump(roots, f, default_flow_style=False, sort_keys=False)
    # with open(out_file, 'w') as f:
    #     yaml.dump(lobes, f, default_flow_style=False, sort_keys=False)

In [ ]:
for scan in scans:
    markers2yml(scan, 'airway')
    markers2yml(scan, 'vessel')